# 지니 Top200 차트

In [94]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

### 1. 인터넷에서 데이터 가져오기

In [95]:
url = 'https://www.genie.co.kr/chart/top200'
req = requests.get(url)
req.text        # The security policy of the connection request is blocked.

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [96]:
# Chrome User-Agent : 나는 로봇이 아니예요...
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'}
req = requests.get(url, headers=header)
soup = BeautifulSoup(req.text, 'html.parser')

### 2. 찾으려고 하는 데이터의 노드 찾기

In [97]:
trs = soup.select('tr.list')        # tag는 tr, class는 list
len(trs)

50

In [98]:
# 개발자 도구에서 찾아주는 Selector
#body-content > div.newest-list > div > table > tbody > tr:nth-child(1)
trs = soup.select('#body-content > div.newest-list > div > table > tbody > tr')
len(trs)

50

### 3. 여러개의 데이터 중 하나를 선택해서 원하는 정보를 추출

In [99]:
tr = trs[12]

In [100]:
# rank
tr.select_one('.number').get_text()

'13\n                                        \n                                    \n1상승\n\n'

In [101]:
rank = tr.select_one('.number').get_text().split('\n')[0]
rank

'13'

In [102]:
try:
    rank = int(tr.select_one('.number').get_text().split('\n')[0])
except:
    rank = 0
rank

13

In [103]:
# 제목
try:        # 일반적인 노래
    title = tr.select_one('.title.ellipsis').string.strip()
except:     # 19금 노래
    title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
title

'신호등'

In [104]:
# artist
artist = tr.select_one('.artist.ellipsis').text.strip()
artist

'이무진'

In [105]:
# album
album = tr.select_one('.albumtitle.ellipsis').text.strip()
album

'신호등'

### 4. 한 페이지내에 있는 정보

- 1) 개별 리스트로 가져오기

In [107]:
rank_list, title_list, artist_list, album_list =  [],[],[],[]
for i, tr in enumerate(trs):
    try:
        rank = int(tr.select_one('.number').get_text().split('\n')[0])
    except:
        rank = 0
        
    try:        # 일반적인 노래
        title = tr.select_one('.title.ellipsis').string.strip()
    except:     # 19금 노래
        title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()   
    artist = tr.select_one('.artist.ellipsis').text.strip()
    album = tr.select_one('.albumtitle.ellipsis').text.strip()
    
    rank_list.append(rank)
    title_list.append(title)
    artist_list.append(artist)
    album_list.append(album)
    

- 2. 줄단위로 딕셔너리 리스트를 만들기

In [108]:
lines = []
for i, tr in enumerate(trs):
    try:
        rank = int(tr.select_one('.number').get_text().split('\n')[0])
    except:
        rank = 0
        
    try:        # 일반적인 노래
        title = tr.select_one('.title.ellipsis').string.strip()
    except:     # 19금 노래
        title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()   
    artist = tr.select_one('.artist.ellipsis').text.strip()
    album = tr.select_one('.albumtitle.ellipsis').text.strip()
    line = {'순위':rank, '제목':title, 
                    '아티스트':artist, '앨범':album}
    lines.append(line)

### 5. 데이터 프레임으로 만들기

In [109]:
df = pd.DataFrame({'순위':rank_list, '제목':title_list, 
                    '아티스트':artist_list, '앨범':album_list})
df.head()

,순위,제목,아티스트,앨범
0,1,봄여름가을겨울 (Still Life),BIGBANG (빅뱅),봄여름가을겨울 (Still Life)
1,2,LOVE DIVE,IVE (아이브),LOVE DIVE
2,3,TOMBOY,(여자)아이들,I NEVER DIE
3,4,Feel My Rhythm,Red Velvet (레드벨벳),'The ReVe Festival 2022 - Feel My Rhythm'
4,5,사랑인가 봐,멜로망스 (MeloMance),사랑인가 봐 (사내맞선 OST 스페셜 트랙)


In [110]:
df = pd.DataFrame(lines)
df.head()

,순위,제목,아티스트,앨범
0,1,봄여름가을겨울 (Still Life),BIGBANG (빅뱅),봄여름가을겨울 (Still Life)
1,2,LOVE DIVE,IVE (아이브),LOVE DIVE
2,3,TOMBOY,(여자)아이들,I NEVER DIE
3,4,Feel My Rhythm,Red Velvet (레드벨벳),'The ReVe Festival 2022 - Feel My Rhythm'
4,5,사랑인가 봐,멜로망스 (MeloMance),사랑인가 봐 (사내맞선 OST 스페셜 트랙)


### 6. 모든 페이지에 대해 가져오기

In [146]:
lines = []
for page in range(1, 5):    # 1~4page
    url = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220415&hh=09&rtm=Y&pg={page}'
    req = requests.get(url, headers=header)
    soup = BeautifulSoup(req.text, 'html.parser')
    trs = soup.select('tr.list')

    for tr in trs:
        try:
            rank = int(tr.select_one('.number').get_text().split('\n')[0])
        except:
            rank = 0
            
        try:        # 일반적인 노래
            title = tr.select_one('.title.ellipsis').string.strip()
        except:     # 19금 노래
            title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()   
        artist = tr.select_one('.artist.ellipsis').text.strip()
        album = tr.select_one('.albumtitle.ellipsis').text.strip()
        line = {'순위':rank, '제목':title, '아티스트':artist, '앨범':album}
        lines.append(line)

In [147]:
df = pd.DataFrame(lines)
df.tail()

,순위,제목,아티스트,앨범
195,196,하루하루,BIGBANG (빅뱅),Stand Up (2008 빅뱅 3rd Mini Album)
196,197,붉은 노을,BIGBANG (빅뱅),REMEMBER
197,198,Saturday Drip,NCT DREAM,Glitch Mode - The 2nd Album
198,199,"너, 너",스트레이 (The Stray),her
199,200,어른,Sondia,나의 아저씨 OST Part. 2 (tvN 수목드라마)


In [148]:
df.to_csv('지니차트Top200.csv', index=False)

In [129]:
lis = soup.find_all('tr')
li = lis[14]     # 1부터 50까지...

<pre>
순위
</pre>

In [130]:
try:
    rank = int(li.select_one('.number').get_text().split('\n')[0])
except:
    rank = 0
rank

'14'

<pre>
제목
</pre>

In [131]:
title = li.select_one('.title.ellipsis').get_text().strip('\n 19금')
title

'abcdefu'

<pre>
가수
</pre>

In [132]:
singer = li.select_one('.artist.ellipsis').get_text().strip('\n ')
singer

'GAYLE'

<pre>
앨범명
</pre>

In [133]:
album = li.select_one('.albumtitle.ellipsis').get_text().strip('\n ')
album

'abcdefu'

<pre>
반복문
</pre>

In [138]:
line = []
for li in lis:      # lis[0]는 Error, try except로 하나를 건너뜀.
    try:
        try:
            rank = int(li.select_one('.number').get_text().split('\n')[0])
        except:
            rank = 0
        title = li.select_one('.title.ellipsis').get_text().strip('\n 19금')
        singer = li.select_one('.artist.ellipsis').get_text()
        album = li.select_one('.albumtitle.ellipsis').get_text()
        line.append({'순위':rank, '제목':title, '가수':singer, '앨범명':album})
    except:
        continue

In [139]:
df = pd.DataFrame(line)
df

,순위,제목,가수,앨범명
0,1,봄여름가을겨울 (Still Life),BIGBANG (빅뱅),봄여름가을겨울 (Still Life)
1,2,LOVE DIVE,IVE (아이브),LOVE DIVE
2,3,TOMBOY,(여자)아이들,I NEVER DIE
3,4,Feel My Rhythm,Red Velvet (레드벨벳),'The ReVe Festival 2022 - Feel My Rhythm'
4,5,사랑인가 봐,멜로망스 (MeloMance),사랑인가 봐 (사내맞선 OST 스페셜 트랙)
5,6,GANADARA (Feat. 아이유),박재범,GANADARA
6,7,듣고 싶을까,MSG워너비 (M.O.M),듣고 싶을까
7,8,취중고백,김민석 (멜로망스),취중고백
8,9,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2
9,10,INVU,태연 (TAEYEON),INVU - The 3rd Album


In [140]:
df.to_csv('지니차트(20220414)Top50.csv', index=False)